## Test your analytics skills by predicting which New York Times blog articles will be the most popular.

Newspapers and online news aggregators like Google News need to understand which news articles will be the most popular, so that they can prioritize the order in which stories appear. In this competition, you will predict the popularity of a set of New York Times blog articles from the time period September 2014-December 2014.

The following screenshot shows an example of the New York Times technology blog "Bits" homepage:

![](https://kaggle2.blob.core.windows.net/competitions/kaggle/4347/media/NYBlogScreenshot_031715.png)

Many blog articles are published each day, and the New York Times has to decide which articles should be featured. In this competition, we challenge you to develop an analytics model that will help the New York Times understand the features of a blog post that make it popular.

#Read in training and testing sets

Originally these data files could be found at this [link](https://www.kaggle.com/c/15-071x-the-analytics-edge-competition-spring-2015/download/NYTimesBlogTest.csv), but the Jupyter R kernal does not support direct read.csv from a url.

The codebook for the two datatsets are nearly identical with only the "Popular" feature missing from the testing set.

##File Descriptions

The data provided for this competition is split into two files:

NYTimesBlogTrain.csv = the training data set. It consists of 6532 articles.
NYTimesBlogTest.csv = the testing data set. It consists of 1870 articles.  

We have also provided a sample submission file, SampleSubmission.csv. This file gives an example of the format of submission files (see the Evaluation page for more information). The data for this competition comes from the New York Times website.

###Variable Descriptions

The dependent variable in this problem is the variable Popular, which labels if an article had 25 or more comments in its online comment section (equal to 1 if it did, and 0 if it did not). The dependent variable is provided in the training data set, but not the testing dataset. This is an important difference from what you are used to - you will not be able to see how well your model does on the test set until you make a submission on Kaggle.

The independent variables consist of 8 pieces of article data available at the time of publication, and a unique identifier:

NewsDesk = the New York Times desk that produced the story (Business, Culture, Foreign, etc.)
SectionName = the section the article appeared in (Opinion, Arts, Technology, etc.)
SubsectionName = the subsection the article appeared in (Education, Small Business, Room for Debate, etc.)
Headline = the title of the article
Snippet = a small portion of the article text
Abstract = a summary of the blog article, written by the New York Times
WordCount = the number of words in the article
PubDate = the publication date, in the format "Year-Month-Day Hour:Minute:Second"
UniqueID = a unique identifier for each article

In [ ]:
nytTrain = read.csv( "NYTimesBlogTrain.csv", stringsAsFactors = FALSE )
nytTest  = read.csv( "NYTimesBlogTest.csv",  stringsAsFactors = FALSE )
nytTest$Popular = rep( 2 , dim( nytTest )[ 1 ] )  ##make the Train and Test sets conformable by adding column to test set with value of '2' as unknonw

In [ ]:
str(nytTrain)
summary(nytTrain)

nytTrain$Headline = gsub("New York","",nytTrain$Headline)
nytTrain$Snippet = gsub("New York","",nytTrain$Headline)
nytTrain$Abstract = gsub("New York","",nytTrain$Headline)

nytTrain$PubDate=strptime(nytTrain$PubDate,'%Y-%m-%d %H:%M:%S')
nytTrain$hour = format(nytTrain$PubDate,'%H')
nytTrain$dow = format(nytTrain$PubDate,'%A')

In [ ]:
##emails$spam = as.factor(emails$spam)
corpus  = Corpus(VectorSource(c(nytTrain$Headline,nytTrain$Snippet,nytTrain$Abstract,nytUNK$Headline,nytUNK$Snippet,nytUNK$Abstract)))
corpus = tm_map(corpus, PlainTextDocument)
corpus = tm_map(corpus, tolower)
corpus = tm_map(corpus, PlainTextDocument)
corpus = tm_map(corpus, removePunctuation)
corpus = tm_map(corpus, removeWords, stopwords("english"))
corpus = tm_map(corpus, stemDocument)
dtm = DocumentTermMatrix(corpus)

In [ ]:
spdtm = removeSparseTerms(dtm, 0.99)
spdtm
nytSparse     = as.data.frame(as.matrix(spdtm))
colnames(nytSparse) = make.names(colnames(nytSparse))

rev(sort(colSums(nytSparse)))[1]
nytSparse$Popular = c(nytTrain$Popular,nytUNK$Popular)

nytSparse$Popular = as.factor(nytSparse$Popular)
nytSparse$WordCount    = nytTrain$WordCount
nytSparse$hour    = as.factor(nytTrain$hour)
nytSparse$dow     = as.factor(nytTrain$dow)

nytTESTset = subset(nytSparse,Popular=='2')
nytSparse = subset(nytSparse,Popular %in% c('0','1'))

In [ ]:
set.seed(123)
split = sample.split(nytSparse$Popular, SplitRatio = 0.7)

train = subset(nytSparse, split==TRUE)
train$Popular = droplevels(train$Popular)
test  = subset(nytSparse, split==FALSE)
test$Popular = droplevels(test$Popular)

In [ ]:
nytLog = glm(Popular ~ .,data=train,family=binomial)


predPopularLog = predict(nytLog, type="response")
x=table(train$Popular, predPopularLog >= 0.5)
sum(diag(x))/nrow(train)


ROCRpred = prediction(predPopularLog, train$Popular)
as.numeric(performance(ROCRpred, "auc")@y.values)



##test set
predPopularLog = predict(nytLog, newdata=test, type="response")
x<-table(test$Popular, predPopularLog >= 0.5)
sum(diag(x))/nrow(test)
ROCRpred = prediction(predPopularLog, test$Popular)
as.numeric(performance(ROCRpred, "auc")@y.values)

In [ ]:
nytCART = rpart(Popular ~ ., data=train, method="class")
prp(nytCART)


predictCART = predict(nytCART, newdata=train, type="class")
table(train$Popular, predictCART)
# Compute accuracy
sum(diag(table(train$Popular, predictCART)))/nrow(train)

predictCART = predict(nytCART, newdata=train)[,2]
ROCRpred = prediction(predictCART, train$Popular)
as.numeric(performance(ROCRpred, "auc")@y.values)


##test set
predictCART = predict(nytCART, newdata=test, type="class")
table(test$Popular, predictCART)
# Compute accuracy
sum(diag(table(test$Popular, predictCART)))/nrow(test)

predictCART = predict(nytCART, newdata=test)[,2]
ROCRpred = prediction(predictCART, test$Popular)
as.numeric(performance(ROCRpred, "auc")@y.values)

In [ ]:
library(randomForest)
set.seed(123)
nytRF = randomForest(Popular ~ ., data=train )

predictRF = predict(nytRF, newdata=train, type="class")
table(train$Popular, predictRF)
# Compute accuracy
sum(diag(table(train$Popular, predictRF)))/nrow(train)

predictRF = predict(nytRF, newdata=train, type='prob')[,2]
ROCRpred = prediction(predictRF, train$Popular)
as.numeric(performance(ROCRpred, "auc")@y.values)


##test
predictRF = predict(nytRF, newdata=test, type="class")
table(test$Popular, predictRF)
# Compute accuracy
sum(diag(table(test$Popular, predictRF)))/nrow(test)

predictRF = predict(nytRF, newdata=test, type='prob')[,2]
ROCRpred = prediction(predictRF, test$Popular)
as.numeric(performance(ROCRpred, "auc")@y.values)

##Evaluation

The evaluation metric for this competition is AUC. The AUC, which we described in Unit 3 when we taught logistic regression, is a commonly used evaluation metric for binary classification problems like this one. The interpretation is that given a random positive observation and negative observation, the AUC gives the proportion of the time you guess which is which correctly. It is less affected by sample balance than accuracy. A perfect model will score an AUC of 1, while random guessing will score an AUC of around 0.5.

##Submission File

For every observation in the test set, submission files should contain two columns: UniqueID and Probability1. The submission should be a csv file. The UniqueID should just be the corresponding UniqueID column from the test dataset. The Probability1 column should be the predicted probability of the outcome 1 according to your model, for that UniqueID. We have provided an example of a submission file, SampleSubmission.csv, which can be found in the Data section on Kaggle.

As an example of how to generate a submission file in R, suppose that your test set probability predictions are called "testPred" and your test data set is called "test". Then you can generate a submission file called "submission.csv" by running the following two lines of code in R (if you copy and paste these lines of code into R, the quotes around submission.csv might not read properly - please delete and re-type the quotes if you get an error):

submission = data.frame(UniqueID = test$UniqueID, Probability1 = testPred)
write.csv(submission, â€œsubmission.csvâ€, row.names=FALSE)

You should then submit the file "submission.csv" by clicking on "Make a Submission" on the Kaggle website.

If you take a look at the file "submission.csv", you should see that the file contains a header and has the following format:

UniqueID,Probability1
6533,0.279672578 
6534,0.695794648 
6535,0.695794648 
6536,0.279672578 
6537,0.554216867 
6538,0.640816327 
6539,0.695794648
etc.

In [ ]:
PredTest = predict(nytRF, newdata=nytUNKSparse, type="response")
MySubmission = data.frame(UniqueID = nytUNK$UniqueID, Probability1 = PredTest)
write.csv(MySubmission, "Submission0.csv", row.names=FALSE)